In [27]:
import wings
from wings.execution import Execution
from wings.data import Data

In [28]:
import os.path
import json
import pandas as pd

In [2]:
SERVER = "https://datascience4all.org/wings-portal-new"
INTERNAL_SERVER = SERVER

USERID = "dkhider"
PASSWORD = "dkhider123"
DOMAIN = "PaleoTS"

CACHEDIR = "cache"

In [3]:
api = wings.ApiClient(server=SERVER, username=USERID, password=PASSWORD)
api.set_domain(DOMAIN)

In [4]:
execution_api = Execution(api)
data_api = Data(api)

In [5]:
execs = execution_api.list_executions_by_page(pattern="WaveletAnalysis*")

In [59]:
full_execs = []
if execs["success"]:
    for ex in execs["rows"]:
        runid = ex["id"].split("#")[1]
        status = ex["runtimeInfo"]["status"]
        startTime = ex["runtimeInfo"]["startTime"]
        if status == "SUCCESS" and startTime > 1670853081:
            print(str(len(full_execs) + 1) + ". Run id: " + runid)
            exec = execution_api.get_run_details(runid)
            full_execs.append(exec)

1. Run id: WaveletAnalysis-42e8a086-2d1e-42aa-b5c1-d17583809c9c
2. Run id: WaveletAnalysis-b50a3f10-638f-4c23-abcb-9914cbd03ea9
3. Run id: WaveletAnalysis-7bb03c53-1e14-44fc-beb7-da985243d654
4. Run id: WaveletAnalysis-811a4a16-478b-4942-ba14-8966d8976c36
5. Run id: WaveletAnalysis-f91290e1-17c5-4ffd-97d9-b474877e5ec5
6. Run id: WaveletAnalysis-ceb1f4eb-afe5-4b31-9212-01b3e3055745
7. Run id: WaveletAnalysis-5b76181a-3586-4005-a1f4-16196f67c08b
8. Run id: WaveletAnalysis-aa3cadd5-d258-48c1-884e-0baf70cc5546
9. Run id: WaveletAnalysis-0de5e852-87af-49f1-b0ee-8e4be03df2be
10. Run id: WaveletAnalysis-c1662039-5b3c-4d29-9c20-038288b193aa
11. Run id: WaveletAnalysis-faacd461-c424-4e75-9eb1-20c4fc9f4f39
12. Run id: WaveletAnalysis-e6a31d23-d9dc-4c0b-b9ec-e0828c0ebd5f
13. Run id: WaveletAnalysis-b7eb007f-4768-49a4-8f94-05f67656ccb5
14. Run id: WaveletAnalysis-a96563b0-7b9d-42fc-9d57-ef1182700d06
15. Run id: WaveletAnalysis-55650abf-2381-46de-916f-431fc9cea2c8
16. Run id: WaveletAnalysis-a4eda6

In [60]:
def get_execution_parameters(exec):
    variables = exec["variables"]
    params = {}
    for var in variables["input"]:
        if var["type"] == 2:
            vbinding = var["binding"]
            if vbinding["type"] == "literal":
                varname = var["id"].split("#")[1]
                #dtype = vbinding["datatype"].split("#")[1]
                value = vbinding["value"]
                params[varname] = value
    return params

In [61]:
def get_execution_files(exec, file_types=[], fetch_datasets_for_variables=[]):
    variables = exec["variables"]
    files = {}
    allvars = []

    for file_type in file_types:
        allvars += variables[file_type]
    for var in allvars:
        if var["type"] == 1:
            vbinding = var["binding"]
            if vbinding["type"] == "uri":
                varname = var["id"].split("#")[1]
                location = None
                dataid = vbinding["id"]
                dataname = dataid.split("#")[1]
                # Download datasets for variables if needed
                if varname in fetch_datasets_for_variables:
                    location = CACHEDIR + "/" + dataname
                    if not os.path.exists(location):
                        data_api.fetch_data(dataid, location)
                    files[varname] = location
    return files

In [75]:
df = None
all_rows = []
for exec in full_execs:
    steps = exec["execution"]["plan"]["steps"]
    cvalues = {}
    for step in steps:
        cname = step["codeBinding"]["codeDirectory"].split("/")[-1]
        cvalues[cname] = True

    params = get_execution_parameters(exec)
    input_files = get_execution_files(exec, ["input"], ["InputTS"])
    output_files = get_execution_files(exec, ["intermediate", "output"], ["ClusterSummary"])

    row = dict(params)
    row.update(input_files)
    row.update(output_files)
    row.update(cvalues)

    if df is None:
        df = pd.DataFrame()
    
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

In [82]:
df.loc[df["cwt"]==True]["ClusterSummary"]

0     cache/ClusterSummary-7aqanxrzccpx27waopwq2bxr7...
2     cache/ClusterSummary-1tprq1cx13q7igz7crvzhu1po...
3     cache/ClusterSummary-1tprq1cx13q7igz7crvzhu1po...
4     cache/ClusterSummary-1tprq1cx13q7igz7crvzhu1po...
5     cache/ClusterSummary-ed3zvvyr71pz92pksjl369stc...
6     cache/ClusterSummary-em46yoh8r224n9ge9ua7zrmnc...
7     cache/ClusterSummary-4sgvr7m8kqhbilm3yfsjqied0...
8     cache/ClusterSummary-eybte1t1x6da0rpyhv5ue6qrb...
16    cache/ClusterSummary-ed3zvvyr71pz92pksjl369stc...
17    cache/ClusterSummary-em46yoh8r224n9ge9ua7zrmnc...
18    cache/ClusterSummary-4sgvr7m8kqhbilm3yfsjqied0...
19    cache/ClusterSummary-eybte1t1x6da0rpyhv5ue6qrb...
20    cache/ClusterSummary-eqrxvui75w5x3yxpn4cdr3xrs...
21    cache/ClusterSummary-3b8x1dp2jahsvp5di4mqtb2gp...
22    cache/ClusterSummary-4sgvr7m8kqhbilm3yfsjqied0...
23    cache/ClusterSummary-eybte1t1x6da0rpyhv5ue6qrb...
24    cache/ClusterSummary-ed3zvvyr71pz92pksjl369stc...
25    cache/ClusterSummary-em46yoh8r224n9ge9ua7z